-----------------
Import needed libraries

In [107]:
import cv2
import numpy as np

------------------
Loading the data

In [108]:
# with mask data
with_mask = np.load('with_mask.npy')

#without mask data
without_mask = np.load('without_mask.npy')

In [109]:
# Shape of the datas
print('Shape of with_mask data : ', with_mask.shape)
print('Shape of without_mask data : ', without_mask.shape)

Shape of with_mask data :  (200, 50, 50, 3)
Shape of without_mask data :  (200, 50, 50, 3)


In [110]:
# Reshaping the data
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200, 50*50*3)

In [111]:
# combine both datas into one np array 
X = np.r_[with_mask, without_mask]

In [122]:
# Generate labels for collected datas
labels = np.zeros(X.shape[0])
labels[0:200] = 1.0

In [142]:
names = {0 : "Mask", 1 : "No Mask"}

------------------
Train our model using SVM

In [133]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [134]:
#split testing data and training data
X_train, X_test, Y_train, Y_test = train_test_split(X, labels, test_size = 0.2)

In [135]:
# Let's reduce the dimensions 
from sklearn.decomposition import PCA

print('dimensions of X_train before using PCA : ', X_train.shape)
pca = PCA(n_components = 3)

X_train = pca.fit_transform(X_train)
print('dimensions of X_train after using PCA : ', X_train.shape)

dimensions of X_train before using PCA :  (320, 7500)
dimensions of X_train after using PCA :  (320, 3)


In [136]:
# Training our model
svm = SVC()
svm.fit(X_train, Y_train)

SVC()

----------------
Test our model


In [137]:
# predict using testing data
X_test = pca.fit_transform(X_test)
Y_predicted = svm.predict(X_test)

In [138]:
# let's find accuracy
print('Accuracy of model : ', accuracy_score(Y_test, Y_predicted))

Accuracy of model :  0.125


---------------------------
Lets check detctions

In [139]:
haar_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [148]:
capture = cv2.VideoCapture(0)
data = []
color_dict = {0 : (0,255,0), 1 : (0, 0, 255)}

while True : 
    flag, img = capture.read()
    if flag : 
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces : 
           
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1, -1)
            face = pca.transform(face)
            prediction = svm.predict(face)
            result = names[int(prediction)]
            cv2.rectangle(img, (x,y), (x+w, y+h), color_dict[int(prediction)], 2)
            cv2.putText(img, names[int(prediction)], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
            #cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
           # cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
            
        
           # print("Result : ", result)
        cv2.imshow('result', img)
        if cv2.waitKey(2) == 27 : 
            break

capture.release()
cv2.destroyAllWindows()